In [ ]:
import cv2
import os
import shutil
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense,Dropout,Flatten,BatchNormalization,MaxPooling2D,Conv2D,Input, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping,ModelCheckpoint
import keras_tuner as kt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

**Dataset Acknowledgment**:
The dataset is the publicly available traffic congestion dataset from Kaggle.

In [ ]:
def extract_frames(video_path, output_folder):

    cap = cv2.VideoCapture(video_path)
    image_index = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame_filename = os.path.join(output_folder, f"frame_{image_index:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        image_index += 1

    cap.release()

In [ ]:
dataset_path = '/content/drive/My Drive/ML PERSONAL/dataset/'

In [ ]:
non_traffic = f'{dataset_path}/non_traffic'
light_traffic = f'{dataset_path}/light_traffic'
moderate_traffic = f'{dataset_path}/moderate_traffic'
heavy_traffic = f'{dataset_path}/heavy_traffic'
congested_traffic = f'{dataset_path}/congested_traffic'

In [ ]:
extract_frames(
    video_path = "/content/drive/My Drive/ML PERSONAL/dataset/Non-traffic related.mp4",
    output_folder = non_traffic
)

In [ ]:
extract_frames(
    video_path = "/content/drive/My Drive/ML PERSONAL/dataset/Light Traffic.mp4",
    output_folder = light_traffic
)

In [ ]:
extract_frames(
    video_path = "/content/drive/My Drive/ML PERSONAL/dataset/Moderate Traffic.mp4",
    output_folder = moderate_traffic
)

In [ ]:
extract_frames(
    video_path = "/content/drive/My Drive/ML PERSONAL/dataset/Heavy Traffic.mp4",
    output_folder = heavy_traffic
)

In [ ]:
extract_frames(
    video_path ="/content/drive/My Drive/ML PERSONAL/dataset/Congested Traffic.mp4",
    output_folder = congested_traffic
)

In [ ]:
train_dir = f'{dataset_path}/train_dir'
valid_dir = f'{dataset_path}/valid_dir'

In [ ]:
class1 = 'non_traffic'
class2 = 'light_traffic'
class3 = 'moderate_traffic'
class4 = 'heavy_traffic'
class5 = 'congested_traffic'

classes = [class1, class2, class3, class4, class5]

for each in classes:
    os.makedirs(os.path.join(train_dir, each), exist_ok=True)
    os.makedirs(os.path.join(valid_dir, each), exist_ok=True)

In [ ]:
def split_data(class_dir, train_ratio, valid_ratio):
    files = os.listdir(class_dir)
    random.shuffle(files)

    num_files = len(files)
    train_split = int(train_ratio * num_files)

    train_files = files[:train_split]
    valid_files = files[train_split:]

    random.shuffle(train_files)
    random.shuffle(valid_files)

    return train_files, valid_files

In [ ]:
train_ratio = 0.85
valid_ratio = 0.15

split_info = {}

for each in classes:

    class_dir = os.path.join(dataset_path, each)
    train, validation = split_data(class_dir, train_ratio, valid_ratio)

    train_size, validation_size = len(train), len(validation)
    split_info[each] = [train_size, validation_size]


    for file in train:
        source_path = os.path.join(class_dir, file)
        destination_path = os.path.join(train_dir, each, file)
        shutil.copy(source_path, destination_path)

    for file in validation:
        source_path = os.path.join(class_dir, file)
        destination_path = os.path.join(valid_dir, each, file)
        shutil.copy(source_path, destination_path)

In [ ]:
img_width, img_height = 300, 300
batch_size = 256

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.1,
    brightness_range=[0.5, 1.25],
    horizontal_flip=True,
    vertical_flip=True
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    seed=55
)

Found 10755 images belonging to 5 classes.


In [ ]:
validation_datagen = ImageDataGenerator()
validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    seed=44
)

Found 1900 images belonging to 5 classes.


In [ ]:
def create_traffic_model(hp):
    base_model = Xception(
        input_shape=(img_width, img_height, 3),
        weights='imagenet',
        include_top=False,
        pooling=hp.Choice('base_model_pooling', values=['avg', 'max'])
    )
    base_model.trainable = False

    i = Input([img_width, img_height, 3])
    x = tf.keras.applications.xception.preprocess_input(i)
    x = base_model(x, training=False)
    x = BatchNormalization()(x)
    x = Dense(
        hp.Int('units_1', min_value=16, max_value=512, step=16),
        activation='relu'
    )(x)
    x = BatchNormalization()(x)
    x = Dense(
        hp.Int('units_2', min_value=16, max_value=512, step=16),
        activation='relu'
    )(x)
    x = Dropout(
        hp.Float('dropout_rate_1', min_value=0.1, max_value=0.6, step=0.1)
    )(x)
    x = Dense(5, activation='softmax')(x)
    model = tf.keras.Model(inputs=i, outputs=x)

    model.compile(
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        optimizer=Adam(
            learning_rate=hp.Choice('learning_rate', values=[0.003, 0.001, 0.0007, 0.0003, 0.0001])
        )
    )
    return model


In [ ]:
tuner = kt.Hyperband(create_traffic_model,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     directory='/content/drive/My Drive/ML PERSONAL/',
                     project_name='traffic_classifier_model')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(
    train_generator,
    steps_per_epoch=(train_generator.samples // batch_size)// 3,
    validation_data=validation_generator,
    validation_steps=(validation_generator.samples // batch_size)  // 3,
    batch_size=batch_size,
    callbacks=[stop_early]
)

Trial 32 Complete [00h 04m 11s]
val_loss: 1.009745717048645

Best val_loss So Far: 0.49334022402763367
Total elapsed time: 00h 46m 10s

Search: Running Trial #33

Value             |Best Value So Far |Hyperparameter
avg               |max               |base_model_pooling
336               |320               |units_1
400               |48                |units_2
0.2               |0.2               |dropout_rate_1
0.001             |0.001             |learning_rate
2                 |4                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


KeyboardInterrupt: 

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 300, 300, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide_1 (TrueDivide)           │ (None, 300, 300, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract_1 (Subtract)                │ (None, 300, 300, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ xception (Functional)                │ (None, 2048)                │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 320)                 │         655,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 320)                 │           1,280 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 48)                  │          15,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 48)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │             245 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,542,285 (82.18 MB)

 Trainable params: 676,069 (2.58 MB)

 Non-trainable params: 20,866,216 (79.60 MB)

In [ ]:
# Define the checkpoint callback
checkpoint_cb = ModelCheckpoint(
    filepath='/content/drive/My Drive/ML PERSONAL/last_checkpoint.keras',  # Filepath format

    save_best_only=False,  # Save the model after every epoch
    save_weights_only=False,  # Save the entire model (architecture + weights)
    monitor='val_loss',  # Metric to monitor
    mode='min',  # Save when 'val_loss' decreases
    verbose=1
)

In [ ]:
# Load the saved model
model = load_model('/content/drive/My Drive/ML PERSONAL/last_checkpoint.keras')

In [ ]:
# Train the model and pass the checkpoint callback
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,  # Steps per epoch
    epochs=51,
    initial_epoch=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,  # Validation steps
    batch_size=batch_size,
    callbacks=[checkpoint_cb]  # Include the ModelCheckpoint callback
)

Epoch 6/51
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9370 - loss: 0.1735
Epoch 6: saving model to /content/drive/My Drive/ML PERSONAL/last_checkpoint.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 268s 5s/step - accuracy: 0.9370 - loss: 0.1735 - val_accuracy: 0.9196 - val_loss: 0.2044
Epoch 7/51
 1/42 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.9102 - loss: 0.2254

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 7: saving model to /content/drive/My Drive/ML PERSONAL/last_checkpoint.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.9102 - loss: 0.2254 - val_accuracy: 0.9074 - val_loss: 0.2590
Epoch 8/51
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9355 - loss: 0.1835
Epoch 8: saving model to /content/drive/My Drive/ML PERSONAL/last_checkpoint.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 261s 5s/step - accuracy: 0.9355 - loss: 0.1836 - val_accuracy: 0.9113 - val_loss: 0.2466
Epoch 9/51
 1/42 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.9219 - loss: 0.2067
Epoch 9: saving model to /content/drive/My Drive/ML PERSONAL/last_checkpoint.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9219 - loss: 0.2067 - val_accuracy: 0.8981 - val_loss: 0.2182
Epoch 10/51
41/42 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.9438 - loss: 0.1678 
Epoch 10: saving model to /content/drive/My Drive/ML PERSONAL/last_checkpoint.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 261s 5s/step - accuracy: 0.9440 - los

**Validation**

In [ ]:
score, acc = model.evaluate(validation_generator)
print('Test Loss =', score)
print('Test Accuracy =', acc)

8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 982ms/step - accuracy: 0.9568 - loss: 0.1206
Test Loss = 0.11640612035989761
Test Accuracy = 0.9605262875556946
